**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
!pip3 install scikit-video

     |████████████████████████████████| 2.3MB 2.9MB/s 


In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization
from keras.layers import Dropout,Flatten

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> The function "act" decide the action to take to move from a state to another. 

> The epsilon help the algorithm to  explore new actions with a probability epsilon, it controls the explore -- exploit paradigm. If we set the epsilon to 0 the agent will always delect the optimal policy and won't explore new policies.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> **position:** the position array is used to store the rate position. It is a grid that contains 2 columns and 2 lines in every border filled with -1, representing places where the rat is not allowed to go. It contains also 1 in the rat position and 0 elsewhere.

> **board:** it is used to describe the environement at each step. It contains the possible rewards with values 0.5 and -1 for the poison and when the rat explore a cell it is set to 0

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0,4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        state = env.reset()
        game_over = False
        win = 0
        lose = 0
        state = env.reset()
        while not game_over:
            # The agent performs an action
            action = agent.learned_act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
            
            # Update the counters
            if reward>0:
                win += reward
            else:
                lose -= reward      

        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [17]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 14.5/18.0. Average score (-3.5)
Win/lose count 9.0/13.0. Average score (-3.75)
Win/lose count 9.5/19.0. Average score (-5.666666666666667)
Win/lose count 9.5/12.0. Average score (-4.875)
Win/lose count 11.0/20.0. Average score (-5.7)
Win/lose count 7.0/18.0. Average score (-6.583333333333333)
Win/lose count 9.5/18.0. Average score (-6.857142857142857)
Win/lose count 11.0/12.0. Average score (-6.125)
Win/lose count 9.0/16.0. Average score (-6.222222222222222)
Win/lose count 9.5/13.0. Average score (-5.95)
Win/lose count 9.5/20.0. Average score (-6.363636363636363)
Win/lose count 10.0/19.0. Average score (-6.583333333333333)
Win/lose count 9.0/16.0. Average score (-6.615384615384615)
Win/lose count 9.0/15.0. Average score (-6.571428571428571)
Win/lose count 10.0/8.0. Average score (-6.0)
Win/lose count 14.0/15.0. Average score (-5.6875)
Win/lose count 16.0/12.0. Average score (-5.117647058823529)
Win/lose count 7.0/9.0. Average score (-4.944444444444445)
Win/lose count 13.

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




We know that:
$$
\mathcal{Q}^{\pi}(s,a)=E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
$$
Therefore:
\begin{equation*}
\mathcal{Q}^{\pi}(s,a)=E_{p^{\pi}}[r(s_0,a_0)+\sum_{t=1}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]
\end{equation*}
\begin{equation*}
\mathcal{Q}^{\pi}(s,a)=E_{p^{\pi}}[r(s_0,a_0)+\gamma\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1}|s_{0}=s,a_{0}=a)]\\
\mathcal{Q}^{\pi}(s,a)=E_{p^{\pi}}[r(s_0,a_0)]+\gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a]
\end{equation*}

Using Markov property:

\begin{equation*}
\mathcal{Q}^{\pi}(s,a)=E_{p^{\pi}}[r(s_{0},a_{0})]+\gamma E_{(s',a')~p(.|s,a)}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1}|s_{1}=s',a_{1}=a')]\\
\end{equation*}

\begin{equation*}
\mathcal{Q}^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s_{0},a_{0})]+\gamma E_{(s',a')~p(.|s,a)}[\sum_{t=0}^{\infty}
\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']\\
\end{equation*}

\begin{equation*}
\mathcal{Q}^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma\mathcal{Q}^{\pi}(s',a')]\\
\end{equation*}

Assuming that an optimal policy $\pi^{\ast}$ exists, the optimal Q function is defined as:
\begin{equation*}
\mathcal{Q}^{\ast}(s,a)=max_{\pi}\mathcal{Q}^{\pi}(s,a)\\
\mathcal{Q}^{\ast}(s,a)=max_{\pi}E_{(s',a')~p(.|s,a)}[r(s,a)+\gamma\mathcal{Q}^{\pi}(s',a')]\\
\mathcal{Q}^{\ast}(s,a)=E_{s'\sim\pi^{\ast}(.|s,a)}[r(s,a)+\gamma\mathcal{Q}^{\pi^{\ast}}(s',a')]\\
\mathcal{Q}^{\ast}(s,a)=E_{s'\sim\pi^{\ast}(.|s,a)}[r(s,a)+\gamma max_{a'}\mathcal{Q}^{\ast}(s',a')]
\end{equation*}

A plausible loss function to minimize is:
\begin{equation*}
\mathcal{L}(\theta)=E_{s'\sim\pi^{\ast}(.|s,a)}((\mathcal{Q}^{\ast}(s,a,\theta)-\mathcal{Q}(s,a,\theta))^{2})\\
\mathcal{L}(\theta)=E_{s'\sim\pi^{\ast}(.|s,a)}((r(s,a)+\gamma max_{a'}\mathcal{Q}^{\ast}(s',a',\theta)-\mathcal{Q}(s,a,\theta))^{2})\\
\mathcal{L}(\theta)=E_{s'\sim\pi^{\ast}(.|s,a)}||r+\gamma max_{a'}\mathcal{Q}(s',a',\theta)-\mathcal{Q}(s,a,\theta)||^{2}
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            self.memory.pop(0)
      

    def random_access(self):
        index = np.random.randint(0, len(self.memory))
        return(self.memory[index])

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s, 0)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            input_states[i] = s_
            target_q[i] = self.model.predict(np.expand_dims(s_, axis = 0))
            
            if game_over_:
                target_q[i][a_] = r_
            else:
                target_q[i][a_] = r_ + self.discount * np.max(self.model.predict(np.expand_dims(n_s_, axis = 0)))
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Dense(64, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Dense(32, activation = "relu"))
        model.add(Flatten())
        model.add(Dense(4))
        ####### FILL IN
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [24]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))










Epoch 000/100 | Loss 0.0149 | Win/lose count 3.0/6.0 (-3.0)
Epoch 001/100 | Loss 0.0586 | Win/lose count 4.0/5.0 (-1.0)
Epoch 002/100 | Loss 0.0086 | Win/lose count 1.0/2.0 (-1.0)
Epoch 003/100 | Loss 0.0089 | Win/lose count 4.0/3.0 (1.0)
Epoch 004/100 | Loss 0.0028 | Win/lose count 4.0/7.0 (-3.0)
Epoch 005/100 | Loss 0.0086 | Win/lose count 6.5/7.0 (-0.5)
Epoch 006/100 | Loss 0.0034 | Win/lose count 2.0/5.0 (-3.0)
Epoch 007/100 | Loss 0.0676 | Win/lose count 7.0/4.0 (3.0)
Epoch 008/100 | Loss 0.0055 | Win/lose count 10.5/5.0 (5.5)
Epoch 009/100 | Loss 0.0038 | Win/lose count 3.5/3.0 (0.5)
Epoch 010/100 | Loss 0.0030 | Win/lose count 10.0/6.0 (4.0)
Epoch 011/100 | Loss 0.0057 | Win/lose count 11.0/2.0 (9.0)
Epoch 012/100 | Loss 0.0592 | Win/lose count 1.5/3.0 (-1.5)
Epoch 013/100 | Loss 0.0059 | Win/lose count 6.0/4.0 (2.0)
Epoch 014/100 | Loss 0.0015 | Win/lose count 4.5/3.0 (1.5)
Epoch 015/100 | Loss 0.0040 | Win/lose count 19.5/2.0 (17.5)
Epoch 016/100 | Loss 0.0658 | Win/l

In [25]:
## Best video
HTML(display_videos('fc_train80.mp4'))

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(32, kernel_size=3, activation="relu"))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [27]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/100 | Loss 0.0002 | Win/lose count 2.5/1.0 (1.5)
Epoch 001/100 | Loss 0.0010 | Win/lose count 4.0/1.0 (3.0)
Epoch 002/100 | Loss 0.0011 | Win/lose count 5.0/5.0 (0.0)
Epoch 003/100 | Loss 0.0080 | Win/lose count 2.5/5.0 (-2.5)
Epoch 004/100 | Loss 0.0051 | Win/lose count 1.5/4.0 (-2.5)
Epoch 005/100 | Loss 0.0023 | Win/lose count 1.0/3.0 (-2.0)
Epoch 006/100 | Loss 0.0074 | Win/lose count 3.5/0 (3.5)
Epoch 007/100 | Loss 0.0038 | Win/lose count 2.0/8.0 (-6.0)
Epoch 008/100 | Loss 0.0020 | Win/lose count 1.0/5.0 (-4.0)
Epoch 009/100 | Loss 0.0038 | Win/lose count 2.5/3.0 (-0.5)
Epoch 010/100 | Loss 0.0388 | Win/lose count 1.0/2.0 (-1.0)
Epoch 011/100 | Loss 0.0039 | Win/lose count 7.5/3.0 (4.5)
Epoch 012/100 | Loss 0.0041 | Win/lose count 7.0/3.0 (4.0)
Epoch 013/100 | Loss 0.0021 | Win/lose count 5.0/3.0 (2.0)
Epoch 014/100 | Loss 0.0603 | Win/lose count 6.5/2.0 (4.5)
Epoch 015/100 | Loss 0.0009 | Win/lose count 6.0/2.0 (4.0)
Epoch 016/100 | Loss 0.0032 | Win/lose count 8.5/5.

In [28]:
HTML(display_videos('cnn_train70.mp4'))

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.8)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=1000, batch_size = 64)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=1000, batch_size = 64)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 52.5/-2.0. Average score (50.5)
Win/lose count 50.5/-1.0. Average score (50.0)
Win/lose count 27.0/0.0. Average score (42.333333333333336)
Win/lose count 24.0/0.0. Average score (37.75)
Win/lose count 53.0/-2.0. Average score (40.4)
Win/lose count 14.0/-1.0. Average score (35.833333333333336)
Win/lose count 46.5/0.0. Average score (37.357142857142854)
Win/lose count 49.0/0.0. Average score (38.8125)
Win/lose count 24.0/0.0. Average score (37.166666666666664)
Win/lose count 32.5/0.0. Average score (36.7)
Win/lose count 46.0/-2.0. Average score (37.36363636363637)
Win/lose count 28.5/0.0. Average score (36.625)
Win/lose count 7.5/0.0. Average score (34.38461538461539)
Win/lose count 27.5/0.0. Average score (33.892857142857146)
Win/lose count 47.0/0.0. Average score (34.766666666666666)
Win/lose count 41.5/0.0. Average score (35.1875)
Win/lose count 51.5/-1.0. Average score (36.088235294117645)
Win/lose count 43.0/0.0. Average score (36.47222222222222)
Win/l

> The CNN performs better than the FC, because in general convolution networks have better performance in dealing with image data which is the case here.

>However our agent suffers from an exploration problem. It doesn't explore much the map, and often gets stuck in corners.

>Regarding the temperature: Changing the temperature means changing the probability in the binomial law. while increasing the temperature the probability to get cheese increases and lowering the probability to get in a poisonous cell. Therefore, we obtain better results but it is biased.




***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,rate_decreasing_epsilon=0.99,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        agent.set_epsilon(agent.epsilon*rate_decreasing_epsilon)
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [0]:
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size=grid_size, max_time=max_time, temperature=temperature)
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
    
    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #error ==> self.position[-2:, :]
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over
    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [35]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/100 | Loss 0.0001 | Win/lose count 2.0/1.0 (1.0)
Epoch 001/100 | Loss 0.0002 | Win/lose count 1.5/2.0 (-0.5)
Epoch 002/100 | Loss 0.0048 | Win/lose count 4.0/3.0 (1.0)
Epoch 003/100 | Loss 0.0022 | Win/lose count 2.5/2.0 (0.5)
Epoch 004/100 | Loss 0.0016 | Win/lose count 2.5/2.0 (0.5)
Epoch 005/100 | Loss 0.0130 | Win/lose count 3.0/3.0 (0.0)
Epoch 006/100 | Loss 0.0042 | Win/lose count 2.0/5.0 (-3.0)
Epoch 007/100 | Loss 0.0028 | Win/lose count 4.0/5.0 (-1.0)
Epoch 008/100 | Loss 0.0020 | Win/lose count 4.5/3.0 (1.5)
Epoch 009/100 | Loss 0.0620 | Win/lose count 3.5/2.0 (1.5)
Epoch 010/100 | Loss 0.0026 | Win/lose count 1.0/3.0 (-2.0)
Epoch 011/100 | Loss 0.0505 | Win/lose count 4.0/4.0 (0.0)
Epoch 012/100 | Loss 0.0055 | Win/lose count 3.5/0 (3.5)
Epoch 013/100 | Loss 0.0049 | Win/lose count 4.0/2.0 (2.0)
Epoch 014/100 | Loss 0.0047 | Win/lose count 4.0/2.0 (2.0)
Epoch 015/100 | Loss 0.0063 | Win/lose count 2.5/2.0 (0.5)
Epoch 016/100 | Loss 0.0005 | Win/lose count 4.0/4.0 (

In [36]:
HTML(display_videos('cnn_train_explore80.mp4'))

In [37]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 18.0/0.0. Average score (18.0)
Win/lose count 8.0/0.0. Average score (13.0)
Win/lose count 6.0/0.0. Average score (10.666666666666666)
Win/lose count 11.5/0.0. Average score (10.875)
Win/lose count 8.0/0.0. Average score (10.3)
Win/lose count 7.5/0.0. Average score (9.833333333333334)
Win/lose count 9.5/0.0. Average score (9.785714285714286)
Win/lose count 3.0/0.0. Average score (8.9375)
Win/lose count 7.5/0.0. Average score (8.777777777777779)
Win/lose count 3.5/0.0. Average score (8.25)
Win/lose count 13.0/0.0. Average score (8.681818181818182)
Win/lose count 19.5/0.0. Average score (9.583333333333334)
Win/lose count 4.5/1.0. Average score (9.115384615384615)
Win/lose count 15.0/0.0. Average score (9.535714285714286)
Win/lose count 2.5/0.0. Average score (9.066666666666666)
Win/lose count 13.0/0.0. Average score (9.3125)
Win/lose count 4.5/0.0. Average score (9.029411764705882)
Win/lose count 3.5/0.0. Average score (8.722222222222221)
Win/lose count 13.0/0.0. Average s

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

**Collect Data**

In [0]:
def collect_data(agent,env,epoch,prefix=''):        
    states = []
    actions = []
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        game_over = False
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            states.append(state)
            actions.append(int(action))

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
        
        # Save as a avi
        if e % 10 == 0:
            env.draw(prefix+str(e))
    return states, actions

# We use the expert agent to collect data

# Evaluation
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_expert = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
agent_expert.load(name_weights='cnn_train_exploremodel.h5',name_model='cnn_train_exploremodel.json')
states, actions = collect_data(agent_expert,env,epochs_train,prefix='collect_data')

states = np.stack(states,axis = 0)
actions = np.array(actions)

**Copycat agent**

In [0]:
class copycat(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(copycat, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(32, (3, 3), activation='relu'))
        #model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [43]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_copycat = copycat(size, lr=.1, epsilon = 0.0, memory_size=2000, batch_size = 32,n_state=3)
agent_copycat.model.fit(states,keras.utils.to_categorical(actions),epochs=100)
agent_copycat.save(name_weights='copycat_model.h5',name_model='copycat_model.json')

Epoch 1/100
20100/20100 [==============================] - 2s 82us/step - loss: 0.1435
Epoch 2/100
20100/20100 [==============================] - 1s 65us/step - loss: 0.1186
Epoch 3/100
20100/20100 [==============================] - 1s 64us/step - loss: 0.1090
Epoch 4/100
20100/20100 [==============================] - 1s 62us/step - loss: 0.1028
Epoch 5/100
20100/20100 [==============================] - 1s 63us/step - loss: 0.0983
Epoch 6/100
20100/20100 [==============================] - 1s 63us/step - loss: 0.0950
Epoch 7/100
20100/20100 [==============================] - 1s 65us/step - loss: 0.0923
Epoch 8/100
20100/20100 [==============================] - 1s 63us/step - loss: 0.0902
Epoch 9/100
20100/20100 [==============================] - 1s 65us/step - loss: 0.0882
Epoch 10/100
20100/20100 [==============================] - 1s 64us/step - loss: 0.0866
Epoch 11/100
20100/20100 [==============================] - 1s 63us/step - loss: 0.0853
Epoch 12/100
20100/20100 [===============

In [44]:
agent_copycat.load(name_weights='copycat_model.h5',name_model='copycat_model.json')
test(agent_copycat,env,epochs_test,prefix='copycat_test')

Win/lose count 7.5/0.0. Average score (7.5)
Win/lose count 3.5/1.0. Average score (5.0)
Win/lose count 6.5/1.0. Average score (5.166666666666667)
Win/lose count 18.5/2.0. Average score (8.0)
Win/lose count 6.0/0.0. Average score (7.6)
Win/lose count 6.5/0.0. Average score (7.416666666666667)
Win/lose count 1.0/0.0. Average score (6.5)
Win/lose count 3.0/0.0. Average score (6.0625)
Win/lose count 21.0/0.0. Average score (7.722222222222222)
Win/lose count 9.0/1.0. Average score (7.75)
Win/lose count 2.5/0.0. Average score (7.2727272727272725)
Win/lose count 9.5/0.0. Average score (7.458333333333333)
Win/lose count 4.5/0.0. Average score (7.230769230769231)
Win/lose count 3.0/0.0. Average score (6.928571428571429)
Win/lose count 5.5/0.0. Average score (6.833333333333333)
Win/lose count 9.0/0.0. Average score (6.96875)
Win/lose count 6.5/0.0. Average score (6.9411764705882355)
Win/lose count 2.0/1.0. Average score (6.611111111111111)
Win/lose count 7.0/0.0. Average score (6.631578947368421

> The copycat agent mimit perfectly the expert agent, and they get similar results